# Programming Exercise 3: Multi-class Classification and Neural Networks

## Giới thiệu

Trong bài tập này, chúng ta sẽ cài đặt mô hình **one-vs-all** logistic regression và neural networks cho nhận dạng chữ số viết tay. Dữ liệu dùng cho bài tập này gồm hai file:

- *ex3data1.mat* là dữ liệu huấn luyện cho bài toán nhận dạng chữ số viết tay.
- *ex3weights.mat* là các trọng số ban đầu cho phần bài tập về neural networks.

## 1. Multi-class Classification

Trong bài tập này, bạn sẽ học cách sử dụng logistic regression và neural networks để nhận dạng chữ số viết tay (từ 0 đến 9). Nhận dạng chữ số viết tay được ứng dụng nhiều trong thực tế, ví dụ: nhận dạng mã bưu điện trên các phong bì hay nhận dạng số tiền trên bank checks.

Trong phần đầu của bài tập này, chúng ta sẽ mở rộng cài đặt logistic regression trong bài tập số 2 và áp dụng phương pháp **one-vs-all** cho phân lớp.

### 1.1 Dữ liệu

File dữ liệu ```ex3data1.mat``` có 5000 ví dụ về chữ số viết tay. File này được lưu ở định dạng ```.mat``` của Octave/Matlab. Để đọc dữ liệu trong python, ta sẽ sử dụng hàm ```scipy.io.loadmat``` (Tham khảo thêm tại [scipy.io](http://docs.scipy.org/doc/scipy/reference/io.html)).

In [1]:
%matplotlib inline
import sys
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat

mat_dict = loadmat('ex3data1.mat')
X = mat_dict['X']
y = mat_dict['y']
m = X.shape[0]

sys.stdout.write('Shape of X: ')
print X.shape
sys.stdout.write('Shape of y: ')
print y.shape

Shape of X: (5000, 400)
Shape of y: (5000, 1)


Đoạn code trên đọc dữ liệu vào các biến ```X``` và ```y``` kiểu ```numpy.ndarray```. Trong đó, ```y``` chứa các nhãn của các example trong dữ liệu. Vì dữ liệu được biến đổi thành dạng tương thích với Octave/Matlab nên chữ số "0" sẽ được biểu diễn bằng nhãn "10" trong tập dữ liệu.

Mỗi ảnh trong tập dữ liệu được lưu dưới dạng 20x20, nên mỗi example là một vector với số chiều là 400. Nếu ta coi mỗi vector là các vector cột thì ```X``` sẽ được biểu diễn dưới dạng sau đây.

$$X=\left[
\begin{array}{c}
-\left(x^{(1)}\right)^T-\\
-\left(x^{(2)}\right)^T-\\
\vdots\\
-\left(x^{(m)}\right)^T-\\
\end{array}
\right]$$

### 1.2  Visualizing the data

Chúng ta sẽ bắt đầu bằng việc visualize dữ liệu. Trong phần này, chúng ta sẽ chọn ngẫu nhiên 100 examples từ dữ liệu và hiển thị các số tương ứng dưới dạng ảnh pixel 20x20 nền xám.

*Note*: Vì đây không phải là nội dung chính của bài học nên tạm thời tôi sẽ bỏ qua phần này để làm các phần quan trọng trước.

In [2]:
print 'Visualizing Data ...'

Visualizing Data ...


### 1.3 Cài đặt logistic regression với vectorization

Để tăng tốc độ huấn luyện trên dữ liệu, trong phần này, chúng ta sẽ cài đặt logistic regression sử dụng kỹ thuật vectorization vì các thao tác trên vector hay ma trận trong python numpy hiệu quả hơn rất nhiều so với sử dụng vòng lặp.

#### 1.3.1 Vectorizing the cost function

Chúng ta sẽ cài đặt hàm cost với kỹ thuật vectorization. Như chúng ta đã biết, trong thuật toán logistic regression, hàm cost function (phiên bản unregularized) được tính bằng công thức sau.

$$J(\theta) =\frac{1}{m}\sum_{i=1}^{m}\left[-y^{(i)}log(h_\theta(x^{(i)})) - (1-y^{(i)})log(1-h_\theta(x^{(i)}))\right]$$

#### 1.3.2 Vectorizing the gradient

Trong logistic regression, gradient của cost function là 1 vector có cùng độ dài với $\theta$ trong đó phần tử $j^{th}$ (với $j=0,1,\cdots,n$ được định nghĩa như sau:

$$\frac{\partial J(\theta)}{\partial \theta_j}=\frac{1}{m}\sum_{i=1}^{m}\left(h_\theta(x^{(i)})-y^{(i)}\right)x_j^{(i)}$$

#### 1.3.3 Regularized logistic regression
Chú ý rằng, hàm cost function cho phiên bản regularization có công thức như sau:
$$J(\theta) =\frac{1}{m}\sum_{i=1}^{m}\left[-y^{(i)}log(h_\theta(x^{(i)})) - (1-y^{(i)})log(1-h_\theta(x^{(i)}))\right] + \frac{\lambda}{2m}\sum_{j=1}^{n}\theta_j^2$$

Chú ý rằng, chúng ta sẽ không áp dụng regularization cho tham số $\theta_0$. Vì thế, gradient cho cost functon là một vector trong đó phần tử thứ $j^{th}$ được định nghĩa như sau:


$$\frac{\partial J(\theta)}{\partial \theta_0}=\frac{1}{m}\sum_{i=1}^{m}\left(h_\theta(x^{(i)}) - y{(i)}\right)x_j^{(i)} \text{   với   } j=0$$

$$\frac{\partial J(\theta)}{\partial \theta_j}=\left(\frac{1}{m}\sum_{i=1}^{m}\left(h_\theta(x^{(i)}) - y{(i)}\right)x_j^{(i)}\right) + \frac{\lambda}{m}\theta_j \text{    với    } j \geq 1$$

In [3]:
# z can be np.ndarray, np.matrix, or scalar
def sigmoid(z):
    return 1 / ( 1 + np.exp(-z) ) 

def cost_function(theta, X, y, _lambda):
    m = y.shape[0]
    J = 0
    hx = sigmoid( np.dot(X, theta) ) # hx.shape m x 1
    y_flat = y.flatten() # for element-wise operators
    J = np.sum( -y_flat * np.log(hx) - (1-y_flat) * np.log(1-hx) )/m
    J += _lambda * (theta[1:] ** 2).sum()/(2*m)
    return J

# theta.shape (n,) (ndim=1)
# X.shape = m x n (ndim = 2)
# y.shape = m x 1 (ndim =2)
# return gradient vector with shape (n,) (ndim=1)
def gradient(theta, X, y, _lambda):
    m = y.shape[0]
    n = theta.shape[0]
    grad = np.zeros( n )
    hx = sigmoid( np.dot(X, theta) ) # hx.shape (m, ) ndim = 1
    hx = hx.reshape( (m, 1) )
    grad = np.dot(X.T, hx - y) / m
    
    grad = grad.flatten()
    temp = np.array(theta)
    temp[0] = 0
    grad += _lambda * temp / m
    
    return grad

### 1.4 One-vs-all Classification
Trong section này, chúng ta sẽ cài đặt mô hình phân lớp one-vs-all bằng cách huấn luyện một số bộ phân lớp với regularized logistic regression. Mỗi bộ phân lớp tương ứng với một class trong tập dữ liệu huấn luyện.

Đầu ra của hàm dưới đây là các tham số cho các bộ phân lớp, và được lưu trong một ma trận $\Theta \in \mathbb{R}^{K\times(N+1)}$. Ở đây, mỗi dòng của ma trận $\Theta$ tương ứng với các tham số của một bộ phân lớp cho lớp tương ứng.

Khi huấn luyện bộ phân lớp cho lớp $k \in \{1,\cdots,K\}$, chúng ta cần một vector chứa các nhãn $y$, trong đó $y_j\in0,1$ để xác định instance thứ *j-th* thuộc lớp $k$ ($y_j=1$) hay hoặc các lớp khác ($y_j=0$). Ta sẽ dùng *logical array* cho công việc này.

In [4]:
from scipy.optimize import fmin_bfgs, fmin

def oneVsAll(X, y, num_labels, _lambda):
    m = X.shape[0] # number of training examples
    n = X.shape[1] # number of features
    all_theta = np.zeros( (num_labels, n+1), dtype=np.float64 )
    # add ones to X data matrix
    X_new = np.concatenate( (np.ones( (m,1) ), X), axis=1 )
    
    for c in xrange(1, num_labels+1):
        initial_theta = np.zeros( n+1, dtype=np.float64 )
        y_lb = np.array( (y == c), dtype=np.int )
        theta = fmin_bfgs(cost_function, initial_theta,
                          fprime=gradient, args=(X_new, y_lb, _lambda),
                          disp=False)
        all_theta[c-1] = theta.reshape( (1, n+1) )
    
    return all_theta

#### 1.4.1 One-vs-all Prediction
Sau khi đã huấn luyện xong bộ phân lớp với phương pháp **one-vs-all**, chúng ta sẽ dự đoán các chữ số viết tay cho mỗi ảnh cho trước. Với mỗi đầu vào, chúng ta sẽ dùng các bộ phân lớp đã học để tính "xác suất" ảnh đó thuộc mỗi lớp, sau đó chúng ta sẽ chọn ra lớp mà bộ phân lớp tương ứng trả về giá trị xác suất lớn nhất.

Trong cài đặt dưới đây, tôi sử dụng hàm ```np.argmax``` để trả về indexes của giá trị lớn nhất trong một chiều cho trước của dữ liệu.

In [5]:
# all_theta.shape = num_labels x (n+1)
# X has shape: m x n
def predictOneVsAll(all_theta, X):
    m = X.shape[0]
    num_labels = all_theta.shape[0]
    p = np.zeros( (m, 1), dtype=np.float32 )
    
    X_new = np.concatenate( (np.ones( (m,1) ), X), axis=1 )
    prob = sigmoid( np.dot(X_new, all_theta.T) )
    p = np.argmax( prob, axis=1 ) + 1
    return p.reshape( (m, 1) )

#### 1.4.2 Tính độ chính xác của mô hình trên dữ liệu huấn luyện
Ta sẽ tính độ chính của mô hình thu được trên dữ liệu huấn luyện. Độ chính xác thu được là 96.48%.

In [6]:
print 'Training One-vs-All Logistic Regression...'
num_labels = 10
_lambda = 0.1
all_theta = oneVsAll(X, y, num_labels, _lambda)

Training One-vs-All Logistic Regression...


Sau khi có các tham số của các bộ phân lớp. Ta sẽ đưa ra dự đoán trên tập dữ liệu training và đưa ra độ chính xác của mô hình.

In [7]:
pred = predictOneVsAll(all_theta, X)
print('Training accuracy (with fmin_bfgs): %s' % 
      ( 100 * (pred == y).mean() ) )

Training accuracy (with fmin_bfgs): 96.48


## 2. Neural Networks

Trong phần trước của bài tập này, chúng ta đã cài đặt **multi-class logistic regression** để nhận dạng chữ số viết tay. Tuy nhiên logistic regression không thể sinh ra các **hypothesis** phức tạp hơn vì nó chỉ là một bộ phân lớp tuyến tính.

Trong phần này, chúng ta sẽ sử dụng **neural networks** để nhận dạng chữ số viết tay sử dụng cùng tập dữ liệu ở trên. Mạng neural sẽ có khả năng biểu diễn các mô hình phức tạp và các hypothesis phi tuyến tính. Trong bài tập này, chúng ta sẽ sử dụng các tham số đã được huấn luyện. Mục tiêu của bài tập là cài đặt thuật toán **feedforward propagation** để sử dụng các trọng số cho việc dự đoán. Trong các bài tập tiếp theo, chúng ta sẽ cài đặt thuật toán **backpropagation** để học các tham số của mạng neural.

### 2.1 Model representation

Hình 2 thể hiện mạng neural chúng ta sử dụng. Mạng neural này bao gồm 3 tầng: tầng input, 1 tầng ẩn, và 1 tầng output.

Các tham số của mạng neural $(\Theta_{(1)},\Theta_{(2)})$ được cung cấp sẵn và lưu trong file ```ex3weights.mat```. Các tham số này có các chiều tương thích với số units của mỗi tầng. Ở đây, chúng ta có 25 units ở tầng thứ 2 (tầng ẩn) và 10 units ở tầng output (tương ứng với 10 digit classes).

<img src="neural_network_model.png" alt="Drawing" style="width:350px;"/>

Đoạn code dưới đây sẽ đọc vào dữ liệu và các tham số từ file.

In [8]:
from scipy.io import loadmat

mat_dict = loadmat('ex3data1.mat')
X = mat_dict['X']
y = mat_dict['y']
m = X.shape[0]

para_dict = loadmat('ex3weights.mat')
Theta1 = para_dict['Theta1']
Theta2 = para_dict['Theta2']

print('Shape of Theta1 and Theta2:')
print Theta1.shape
print Theta2.shape

Shape of Theta1 and Theta2:
(25, 401)
(10, 26)


### 2.2 Thuật toán Feedforward Propagation và dự đoán

Trong phần này, chúng ta sẽ cài đặt thuật toán **Feedforward Propagation** để đưa ra dự đoán nhãn của mỗi ảnh trong dữ liệu.

In [9]:
# X: 5000 * 400
# Theta1: 25 x 401
# Theta2: 10 * 26
def predict(Theta1, Theta2, X):
    m = X.shape[0]
    X_new = np.concatenate( (np.ones( (m,1) ), X), axis=1 ) # 5000 x 401
    A2 = sigmoid( np.dot(X_new, Theta1.T) ) # 5000 x 25
    A2 = np.concatenate( ( np.ones( (m,1) ), A2 ), axis=1 ) # 5000 x 26
    A3 = sigmoid( np.dot(A2, Theta2.T) ) # 5000 x 10
    p = np.argmax( A3, axis=1 ) + 1
    return p.reshape( (m, 1) )    

Áp dụng hàm ```predict``` đã cài đặt ở trên, chúng ta sẽ tính độ chính xác của mô hình trên dữ liệu huấn luyện. Độ chính xác trên dữ liệu huấn luyện là 97.52%.

In [10]:
pred = predict(Theta1, Theta2, X)
print('Training accuracy (with neural networks): %s' % 
      ( 100 * (pred == y).mean() ) )

Training accuracy (with neural networks): 97.52
